# 1. Importing Libraries

In [1]:
import os

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# 2. Importing the dataset

In [2]:
PROJECT_DIR = r"D:\flight-price-pred-sagemaker"
DATA_DIR = "data"

In [3]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [4]:
flights = get_data("flight_price")

In [5]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset cointains 10,683 row and 11 features
- Columns `Route` and `Total_stops` have missing values

# 3. Preliminary Analysis

### 3.1 Check Data Types of all features

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

### 3.2 Check for Duplicates

In [8]:
flights.duplicated().sum()

220

In [9]:
# Check the duplicated rows

(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination"])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- All have string/object data type except `Price` features
- `Date_of_Journey`, `Dep_Time`, `Arrival_Time` should be in datetime datatype.
- The type of `Duration` and `Total_Stops` is mixed, it should be of numeric type.
- There are 220 duplicates. These should be removed

# 4. Detailed Analysis

### Airline

In [10]:
flights["Airline"].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Convert `Vistara Premium economy` to `Vistara` and `Jet Airways Business` to `Jet Airways`

In [11]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### Date_of_Journey

In [12]:
flights["Date_of_Journey"]

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [13]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [14]:
flights["Source"].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [15]:
flights["Destination"].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_Time

In [16]:
flights["Dep_Time"]

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [17]:
(
    flights
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [18]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time

In [19]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [20]:
pd.to_datetime(flights.Arrival_Time).dt.time

0        01:10:00
1        13:15:00
2        04:25:00
3        23:30:00
4        21:35:00
           ...   
10678    22:25:00
10679    23:20:00
10680    11:20:00
10681    14:10:00
10682    19:15:00
Name: Arrival_Time, Length: 10683, dtype: object

In [21]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    
)

0        22 Mar
2        10 Jun
6        13 Mar
7        02 Mar
8        13 Mar
          ...  
10666    13 Jun
10667    13 Mar
10672    28 Jun
10673    28 May
10674    13 Mar
Name: Arrival_Time, Length: 4335, dtype: object

### Duration

In [22]:
flights.Duration 

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [23]:
# Convert this to minutes
(
    flights
    .Duration
    .drop(index = [6474])
    .str.split(" ",expand = True)
    .set_axis(["hour","minute"], axis = 1)
    .assign(
        hour = lambda df_ : (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_ : (
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [24]:
flights.iloc[[6474]] # Delete this row

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


### Total_stops

In [25]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [26]:
(
    flights
    .Total_Stops
    .drop(index = [9039])
    .str.replace("non-stop","0")
    .str.replace(" stops?", "",regex = True)
    .astype(int)
    .unique()
)

array([0, 2, 1, 3, 4])

### Additional_Info

In [27]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [28]:
(
    flights
    .Additional_Info
    .str.replace("Info", "info")
    .unique()
)

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

# 5. Cleaning Operations

In [29]:
def convert_to_min(ser):
    df = (
        ser
        .str.split(" ",expand = True)
        .set_axis(["hour","minute"], axis = 1)
        .assign(
                hour = lambda df_ : (
                    df_
                    .hour
                    .str.replace("h", "")
                    .astype(int)
                    .mul(60)
                ),
                minute = lambda df_ : (
                    df_
                    .minute
                    .str.replace("m","")
                    .fillna("0")
                    .astype(int)
                )
            )
            .sum(axis=1)
         )
    return df


In [30]:
def clean_data(df):
    df = df.drop(index = [6474,9039])
    df = df.drop_duplicates()
    df = df.assign(
            **{
                col: df[col].str.strip()
                for col in df.select_dtypes(include = "O").columns # O --> dtype = Object
            }
        ) # Remove the striped values in the object data type
    df = df.rename(columns = str.lower) # Convert the uppercase to lowercase columns
    df = df.assign(
        airline = lambda df_ : (
            df_
            .airline
            .str.replace(" Premium economy", "")
            .str.replace(" Business", "")
            .str.title()
        ),# Convert Vistara Premium economy to Vistara and Jet Airways Business to Jet Airways
        date_of_journey = lambda df_ : pd.to_datetime(df_.date_of_journey, dayfirst=True), #Convert object dataframe to date time dataframe for feature "date_of_journey"
        dep_time = lambda df_ : pd.to_datetime(df_.dep_time).dt.time,  # Convert Departure time to HH:MM:SS
        arrival_time = lambda df_ : pd.to_datetime(df_.arrival_time).dt.time,  # Convert Arrival time to HH:MM:SS
        # duration = lambda df_ : convert_to_min(df_.duration)
        duration = lambda df_ : df_.duration.pipe(convert_to_min), # Convert the HH:MM to minutes 
        total_stops = lambda df_ : (
            df_
            .total_stops
            .str.replace("non-stop","0")
            .str.replace(" stops?", "",regex = True)
            .astype(int)
        ),
        additional_info = lambda df_ : (
            df_
            .additional_info
            .str.replace("No info", "No Info")    
        )
    )
    df = df.drop(columns = "route")
    return df

In [31]:
flights_cleaned = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0,No Info,12648


# 6. Split the Data

In [32]:
flights_final = flights_cleaned.sample(10000)

In [33]:
X = flights_final.drop(columns = "price") # Input feature
y = flights_final.price.copy() # Target feature

In [34]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size = 0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6400, 9) (6400,)
(1600, 9) (1600,)
(2000, 9) (2000,)


# 7. Export the subsets

In [35]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    X.join(y).to_csv(file_path, index = False)
    return pd.read_csv(file_path).head()

In [36]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-06,Delhi,Cochin,20:25:00,01:30:00,305,1,No Info,5054
1,Indigo,2019-06-03,Kolkata,Banglore,09:35:00,12:10:00,155,0,No Info,4804
2,Air Asia,2019-03-24,Banglore,New Delhi,23:25:00,02:10:00,165,0,No Info,4482
3,Goair,2019-05-18,Kolkata,Banglore,23:30:00,07:45:00,495,1,No Info,7335
4,Indigo,2019-05-18,Delhi,Cochin,16:00:00,21:00:00,300,1,No Info,6069


In [37]:
export_data(X_val, y_val, "validation")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Delhi,Cochin,11:45:00,04:25:00,1000,2,No Info,13882
1,Indigo,2019-06-15,Delhi,Cochin,08:35:00,22:30:00,835,1,No Info,6442
2,Indigo,2019-05-06,Mumbai,Hyderabad,06:25:00,07:55:00,90,0,No Info,4049
3,Air India,2019-05-09,Kolkata,Banglore,09:25:00,07:55:00,1350,1,No Info,8366
4,Indigo,2019-03-18,Kolkata,Banglore,10:10:00,12:55:00,165,0,No Info,4462


In [38]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-03-18,Delhi,Cochin,18:05:00,21:10:00,185,0,No Info,4729
1,Air India,2019-03-01,Banglore,New Delhi,05:50:00,22:10:00,980,2,Change airports,13366
2,Jet Airways,2019-05-15,Kolkata,Banglore,20:25:00,09:45:00,800,1,In-flight meal not included,12384
3,Multiple Carriers,2019-06-03,Delhi,Cochin,08:30:00,19:15:00,645,1,No Info,7677
4,Air India,2019-03-18,Banglore,New Delhi,17:00:00,19:45:00,165,0,No Info,4441
